In [1]:
import torch
import lightgbm as lgb

import os
import sys
PROJECT_DIR = os.path.abspath("../..")
sys.path.append(PROJECT_DIR)

from basicts.utils import load_pkl
from basicts.data import TimeSeriesForecastingDataset
from basicts.metrics import masked_mae, masked_rmse, masked_mape
from basicts.data import SCALER_REGISTRY


## Construct hyper parameters

In [2]:
# construct configs
dataset_name = "PEMS08"
input_len = 12
output_len = 12
gpu_num = 1
null_val = 0.0
train_data_dir = "datasets/" + dataset_name
rescale = True
batch_size = 128 # only used for collecting data

# lgm params
params = {
    'boosting_type': 'gbdt',
    'objective': 'regression',
    'metric': 'l2',
    'num_leaves': 31,
    'learning_rate': 0.05,
    'feature_fraction': 0.9,
    'bagging_fraction': 0.8,
    'bagging_freq': 5,
    'verbose': 0
}

## Construct datasets

In [3]:
# construct dataset
data_file_path = PROJECT_DIR + "/{0}/data_in_{1}_out_{2}_rescale_{3}.pkl".format(train_data_dir, input_len, output_len, rescale)
index_file_path = PROJECT_DIR + "/{0}/index_in_{1}_out_{2}_rescale_{3}.pkl".format(train_data_dir, input_len, output_len, rescale)

train_set = TimeSeriesForecastingDataset(data_file_path, index_file_path, mode="train")
train_loader = torch.utils.data.DataLoader(train_set, batch_size=batch_size, shuffle=True)

valid_set = TimeSeriesForecastingDataset(data_file_path, index_file_path, mode="valid")
valid_loader = torch.utils.data.DataLoader(valid_set, batch_size=batch_size, shuffle=False)

test_set = TimeSeriesForecastingDataset(data_file_path, index_file_path, mode="test")
test_loader = torch.utils.data.DataLoader(test_set, batch_size=batch_size, shuffle=False)

In [4]:
# training & validation
Xs_train = []
Ys_train = []
Xs_valid = []
Ys_valid = []
Xs_test = []
Ys_test = []

for i, (data, target) in enumerate(train_loader):
    B, L, N, C = data.shape
    data = data.transpose(1, 2).reshape(B*N, L, C)[:, :, 0]
    target = target.transpose(1, 2).reshape(B*N, L, C)[:, :, 0]
    Xs_train.append(data)
    Ys_train.append(target)

for i, (data, target) in enumerate(valid_loader):
    B, L, N, C = data.shape
    data = data.transpose(1, 2).reshape(B*N, L, C)[:, :, 0]
    target = target.transpose(1, 2).reshape(B*N, L, C)[:, :, 0]
    Xs_valid.append(data)
    Ys_valid.append(target)

for i, (data, target) in enumerate(test_loader):
    B, L, N, C = data.shape
    data = data.transpose(1, 2).reshape(B*N, L, C)[:, :, 0]
    target = target.transpose(1, 2).reshape(B*N, L, C)[:, :, 0]
    Xs_test.append(data)
    Ys_test.append(target)

Xs_train = torch.cat(Xs_train, dim=0).numpy()
Ys_train = torch.cat(Ys_train, dim=0).numpy()
Xs_valid = torch.cat(Xs_valid, dim=0).numpy()
Ys_valid = torch.cat(Ys_valid, dim=0).numpy()
Xs_test = torch.cat(Xs_test, dim=0).numpy()
Ys_test = torch.cat(Ys_test, dim=0).numpy()


## Direct Multi-step Forecasting (Train)

In [ ]:
# direct forecasting
from sklearn.multioutput import MultiOutputRegressor
model = MultiOutputRegressor(lgb.LGBMRegressor(), n_jobs = -1)
model.fit(Xs_train, Ys_train)

## Direct Multi-step Forecasting (Test)

In [ ]:
# inference
preds_test = model.predict(Xs_test)
# rescale
scaler = load_pkl(PROJECT_DIR + "/{0}/scaler_in_{1}_out_{2}_rescale_{3}.pkl".format(train_data_dir, input_len, output_len, rescale))
preds_test = torch.Tensor(preds_test).view(-1, N, output_len).transpose(1, 2).unsqueeze(-1)
Ys_test = torch.Tensor(Ys_test).view(-1, N, output_len).transpose(1, 2).unsqueeze(-1)
prediction = SCALER_REGISTRY.get(scaler["func"])(preds_test, **scaler["args"])
real_value = SCALER_REGISTRY.get(scaler["func"])(Ys_test, **scaler["args"])
# print results
print("MAE: ", masked_mae(prediction, real_value, null_val).item())
print("RMSE: ", masked_rmse(prediction, real_value, null_val).item())
print("MAPE: {:.2f}%".format(masked_mape(prediction, real_value, null_val) * 100))